# Deep Learning 2

In this practical session, we are going to train a deep neural network to classify articles of clothing based solely on a picture of said article and nothing more. Specifically, we are going to use the [Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) data set, which is conveniently built into Keras:

In [ ]:
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

Let's have a look at some of its samples.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

print('Training set shape: {}'.format(x_train.shape))
print('Test set shape: {}'.format(x_test.shape))

sample = x_train[np.random.randint(x_train.shape[0])]
plt.imshow(sample, cmap='gray_r')
plt.axis('off')
plt.show()

Fashion-MNIST consists of 70,000 grayscale images 28x28 pixels in size, generated from various articles of clothing sold by [Zalando](https://www.zalando.be) (no, I am not being paid extra for this product placement).

## Building a classifier

We are now going to construct a machine learning model for classifying Fashion-MNIST samples.

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(250, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.compile(optimizer=tf.keras.optimizers.Adam(),
          loss=tf.keras.losses.CategoricalCrossentropy(),
          metrics=[tf.keras.metrics.CategoricalAccuracy()])

We can get a nice summary of the model architecture using the Keras function `summary`:

In [ ]:
print(model.summary())

As you can see, this model has 421,576 trainable parameters. This might seem like a lot, but modern DNNs easily have several millions of parameters. Except for its low number of parameters, this architecture is quite typical of modern neural networks for image recognition. It consists of alternating convolutional and pooling layers followed by a flattening operation which turns the high-dimensional tensors into a single vector. This vector is then fed to a few fully-connected layers, i.e. a regular MLP. The convolutional and pooling layers can be seen as *feature extractors*: these layers learn to extract salient information from the input which can be easily interpreted by the MLP at the end. If you have `pydot` and `graphviz` installed, Keras also provides visualization tools which allow us to graph the model:

In [ ]:
from PIL import Image
tf.keras.utils.plot_model(model, to_file='model.png')

im = Image.open('model.png')
im

We'd now like to actually train this model on Fashion-MNIST. To do so, there are a few technical details that need to be taken care of:

1. The labels in `y_train` and `y_test` are not yet one-hot encoded.
2. The samples are not normalized.
3. The training and test samples `x_train` and `x_test` must be 4-dimensional in order to be compatible with the convolutional layers.

We take care of these details and then fit the model:

In [ ]:
x_train = np.expand_dims(x_train, -1).astype(np.float32) / 255
x_test = np.expand_dims(x_test, -1).astype(np.float32) / 255

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

history = model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_split=.2)

Note that the process of training this model takes a little bit of time (five minutes on my system). We can then validate the model on the test set:

In [ ]:
scores = model.evaluate(x_test, y_test, batch_size=50)
print('Loss: {}'.format(scores[0]))
print('Accuracy: {}'.format(scores[1]))

You should get approximately 88% accuracy.

Keras has also returned a `History` object, which allows us to plot the performance of the model over time:

In [ ]:
plt.plot(history.history['categorical_accuracy'])
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

From this plot, you can see the evolution of model accuracy as training progresses. Typical behavior here is an early, quick increase in accuracy which tapers off after a while and eventually starts to randomly fluctuate around an average value. In case of overfitting, you will notice the validation accuracy start to consistently drop off, but that should not happen here. Because of the limited number of epochs in our experiment, however, we might not observe that behavior here as the model may not have fully converged.

We can now use this model to annotate images:

In [ ]:
idx = np.random.randint(x_test.shape[0])
sample = x_test[idx]
index = np.argmax(model.predict(np.expand_dims(sample, 0)))
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal',
           'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

label = classes[index]
plt.title(label)
plt.imshow(sample.squeeze(), cmap='gray_r')
plt.axis('off')
plt.show()

## A word of warning

In recent years, deep learning has made possible vast advances in certain technological applications (particularly image and speech recognition) that nobody had thought possible. Yet, these advances come with a big caveat that you must keep in mind whenever you're considering deep learning for any particular application:

>**Deep learning is not magic.**

This may sound trivial, but we find time and again that this caveat is ignored both by the media as well as established academic researchers in the field. Moreover, much deep learning research that is adopted into practical applications can be considered morally dubious or pseudo-scientific. Some examples:

* Contrary to current popular practice in DL, [human emotions cannot be readily identified via facial expressions alone](https://www.psychologicalscience.org/publications/emotional-expressions-reconsidered-challenges-to-inferring-emotion-from-human-facial-movements.html), yet there is much research on inferring emotions from facial images. Facial recognition is also used, for instance, to infer [criminal behavior](https://confilegal.com/wp-content/uploads/2016/11/ESTUDIO-UNIVERSIDAD-DE-JIAO-TONG-SHANGHAI.pdf) or [sexual orientation](https://www.gsb.stanford.edu/faculty-research/publications/deep-neural-networks-are-more-accurate-humans-detecting-sexual), even though it should be obvious that facial images contain almost no relevant information to make these sorts of predictions. The high accuracy scores reported in such papers can almost always be explained by biases in the data sets or other methodological errors. This phenomenon has been called "digital phrenology" by analogy with [classical phrenology](https://www.wikiwand.com/en/Phrenology).
* Inferring sexual orientation from facial images can also be seen as an example not just of digital phrenology but also of morally dubious applications of DL. Considering that being gay is illegal in many countries and some even [impose the death penalty](https://edition.cnn.com/2019/10/12/africa/uganda-wants-death-penalty-gay-sex-intl/index.html), such technology could easily be abused in mass surveillance by oppressive regimes. In fact, [China is already doing this](https://www.nytimes.com/2019/04/14/technology/china-surveillance-artificial-intelligence-racial-profiling.html) to track and detain members of certain minority populations, such as the Uyghur ([Wang et al. 2016](https://link.springer.com/chapter/10.1007/978-3-319-46654-5_20); [Yaermaimaiti 2018](https://www.tandfonline.com/doi/abs/10.1080/09720502.2018.1493029)).
* [Deep learning does not always outperform "classical" ML](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0194889). Researchers are often unaware of this because they implicitly assume DL will be better than any classical method, so they don't even compare against the classical methods anymore.

In general, before attempting to use DL for a certain problem, ask yourself the following questions:

1. Is DL even necessary for this problem? Will a simple linear regression or SVM not do the trick? Always compare against these simple baselines!
2. Are there potential problems in the data set which might artificially inflate the accuracy scores? Beware of [confounding variables](https://arxiv.org/abs/1807.00431).
3. Is the problem we're trying to solve even solvable with the data we have? Beware of digital phrenology and other pseudo-scientific nonsense. Know that you cannot just throw data at a DL system and expect it to be able to actually solve whatever problem you're facing. Know also that the DL system will do its very best to make you *think* it's solved the problem when it very well might not have.
4. Is what we're doing ethical? Predicting sensitive attributes such as race, sexual orientation, income etc. from easily obtainable data such as faces is not only pseudo-scientific; it can also be very harmful. Consider the societal implications of what you're doing, since these may be vast in the age of big data.

Further resources:

* [Excavating AI: The Politics of Images in Machine Learning Training Sets](https://www.excavating.ai/)
* [Kate Crawford: The Trouble with Bias](https://www.youtube.com/watch?v=fMym_BKWQzk).
* [Ali Rahimi: NIPS 2017 Test-of-time Award Speech](https://www.youtube.com/watch?v=Qi1Yry33TQE).
* [Cathy O’Neill: Weapons of Math Destruction](https://weaponsofmathdestructionbook.com/).
* [Sigal Samuel: Some AI just shouldn’t exist](https://bit.ly/2GttUYW).

## Exercise 1

Given that [MLPs are universal function approximators](https://www.wikiwand.com/en/Universal_approximation_theorem), why do we bother with "special" operations like the convolutional and pooling layers? Why don't we just use one big MLP instead? By the UAP theorem, such a network should be equally capable (if not more so) to classify Fashion-MNIST.

## Exercise 2

Machine learning models are supposed to be robust to all sorts of naturally occuring noise. How robust is the model we trained to Gaussian noise? Test this by computing

$$\begin{aligned}
    v &= x + \varepsilon z
\end{aligned}$$

where $x$ is a sample from the test set, $\varepsilon$ is a scalar and $z$ is standard Gaussian noise. For different values of $\varepsilon$, compute the accuracy on the test set when the samples are corrupted by Gaussian noise. Do this by corrupting the original test samples from `x_test` and normalizing them using the mean and variance from `x_train`. You can limit $\varepsilon$ to the interval $[0, 1]$.

**Remark.** Adding noise may cause some pixel values to go out of the range the network was trained on. To avoid this, [clip the values](https://docs.scipy.org/doc/numpy/reference/generated/numpy.clip.html) to the original interval after adding the noise.

## Exercise 3

TensorFlow 2 allows us to easily compute the gradients of the loss function for any input we desire, as follows:

In [ ]:
idx = np.random.randint(x_test.shape[0])
sample = np.expand_dims(x_test[idx], 0)
label = np.expand_dims(y_test[idx], 0)
ce_loss = tf.keras.losses.CategoricalCrossentropy()
input_var = tf.Variable(sample, dtype=tf.float32)
with tf.GradientTape() as tape:
    prediction = model(input_var)
    loss = ce_loss(label, prediction)
grad_values = tape.gradient(loss, input_var)[0]

The array `grad_values` now contains the gradients of the loss function with respect to the `sample` which we randomly chose from the test set. What happens when we take such a random sample from the test set and add a scaled version of the signs of the gradients to it? That is, we compute

$$
    \tilde{x} = x + \varepsilon \cdot \mathrm{sgn}(g)
$$

where $x$ is a sample from the test set, $\varepsilon > 0$ is a real scalar and $g$ is the vector of gradients of the model at $x$. Visualize $x$ and $\tilde{x}$ for several random samples from the test set and compute their model classifications for different values of $\varepsilon$. What do you observe? Can you draw any conclusions from this?

## Exercise 4

The [CIFAR-10 data set](https://www.cs.toronto.edu/~kriz/cifar.html) consists of 60.000 RGB color images 32x32 pixels in size. Try to construct a convolutional neural network for this problem that achieves as high an accuracy as you can. State-of-the-art accuracy on CIFAR-10 at the time of this writing is about 98% (see [this paper](https://arxiv.org/abs/1805.09501)).

## Exercise 5

Convolutional neural networks were originally proposed for image processing, but they can actually be used for more than that. As an example, *character-level language CNNs* are designed for processing sequences of text. If the length of these sequences is bounded by a constant, then recurrent architectures are not necessary; it's possible to just use a regular feedforward CNN.

Keras also has the IMDB movie review data set built in. This data set contains reviews from IMDB (encoded as lists of integers) together with a label (positive or negative). Construct a CNN for classification of these movie reviews. **Tip:** have a look at the [Embedding layer](https://keras.io/layers/embeddings/).

If you want to play around with recurrent networks, you can compare the CNN you built here to an RNN. Keras has [great support](https://keras.io/layers/recurrent/) for recurrent architectures. Normally an LSTM should be sufficient for this task, but feel free to experiment.